<a href="https://colab.research.google.com/github/nithishk20/Machine-Learning/blob/main/ML_4Ques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
d=pd.read_csv('/content/residential_properties.csv')
d

,Size (sq ft),Bedrooms,Bathrooms,Location,Year Built,Garage Size,Distance to School (mi),Selling Price (k$)
0,1800,3,2,Downtown,1990,1,0.5,250
1,2200,4,3,Suburban,2005,2,1.2,400
2,1200,2,1,Rural,1985,0,3.6,150
3,3000,5,4,Downtown,2010,3,0.8,700
4,2500,4,3,Suburban,2000,2,2.1,500
5,1400,2,1,Rural,1970,1,2.7,175
6,2000,3,2,Downtown,2008,2,0.3,350
7,1800,3,2,Suburban,1995,1,1.8,300
8,1600,2,1,Rural,1980,0,4.3,125
9,2800,4,3,Downtown,2015,3,0.6,650


In [2]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Size (sq ft)             19 non-null     int64  
 1   Bedrooms                 19 non-null     int64  
 2   Bathrooms                19 non-null     int64  
 3   Location                 19 non-null     object 
 4   Year Built               19 non-null     int64  
 5   Garage Size              19 non-null     int64  
 6   Distance to School (mi)  19 non-null     float64
 7   Selling Price (k$)       19 non-null     int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 1.3+ KB


In [3]:
d.columns=d.columns.str.replace(' ','_')
d.columns

Index(['Size_(sq_ft)', 'Bedrooms', 'Bathrooms', 'Location', 'Year_Built',
       'Garage_Size', 'Distance_to_School_(mi)', 'Selling_Price_(k$)'],
      dtype='object')

In [4]:
df=d.select_dtypes(exclude=['object'])

In [5]:
df.describe()

,Size_(sq_ft),Bedrooms,Bathrooms,Year_Built,Garage_Size,Distance_to_School_(mi),Selling_Price_(k$)
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,2142.105263,3.210526,2.263158,1997.105263,1.526316,1.968421,405.789474
std,696.272448,1.182227,1.097578,18.438772,1.073334,1.459091,242.837755
min,1000.000000,1.000000,1.000000,1950.000000,0.000000,0.300000,100.000000
25%,1700.000000,2.000000,1.000000,1987.500000,1.000000,0.750000,187.500000
50%,2000.000000,3.000000,2.000000,2003.000000,2.000000,1.500000,375.000000
75%,2650.000000,4.000000,3.000000,2010.000000,2.000000,2.950000,575.000000
max,3400.000000,5.000000,4.000000,2020.000000,3.000000,5.200000,900.000000


In [6]:
from sklearn import preprocessing
df1=df.copy()
#scaler=preprocessing.MinMaxScaler()->default feature_range=(0,1)
scalar=preprocessing.MinMaxScaler(feature_range=(0,1))
df1.iloc[:,:]=scalar.fit_transform(df1)

In [7]:
x=df1.drop('Selling_Price_(k$)',axis=1)
y=df1['Selling_Price_(k$)']

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=2)

In [9]:
#2.training SVR model, rbf kernel
from sklearn.svm import SVR
model=SVR(kernel='rbf')
model.fit(x_train,y_train)

SVR()

In [10]:
pre=model.predict(x_test)

In [11]:
from sklearn import metrics
metrics.mean_absolute_error(y_test,pre)

0.05428990240233164

In [12]:
#3.MSE
metrics.mean_squared_error(y_test,pre)

0.0052382737999676

In [13]:
import numpy as np
rmse=np.sqrt(metrics.mean_squared_error(y_test,pre))
print(rmse)

0.07237592002847079


In [14]:
model.score(x_test,y_test)

0.9006668079413551

In [15]:
model.score(x_train,y_train)

0.9358902779279757

In [16]:
#4.#Regularization using Cross validation
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
import numpy as np
# Set up Ridge regression with cross-validation
ridge = RidgeCV(alphas=np.logspace(-10, 10, 21), cv=5)
# Perform cross-validation and compute mean squared error
mse = -cross_val_score(ridge, x, y, scoring='neg_mean_squared_error', cv=5)
# Print mean squared error for each alpha
for alpha, mse_val in zip(ridge.alphas, mse):
    print("Alpha: {:.2e} -- Mean squared error: {:.2f}".format(alpha, mse_val))

Alpha: 1.00e-10 -- Mean squared error: 0.01
Alpha: 1.00e-09 -- Mean squared error: 0.00
Alpha: 1.00e-08 -- Mean squared error: 0.01
Alpha: 1.00e-07 -- Mean squared error: 0.01
Alpha: 1.00e-06 -- Mean squared error: 0.00


In [17]:
#4.Lasso regularization with cross-validation
lasso = LassoCV(alphas=np.logspace(-10, 10, 21), cv=5)
# Perform cross-validation and compute mean squared error
mse = -cross_val_score(lasso, x, y, scoring='neg_mean_squared_error', cv=5)
# Print mean squared error for each alpha
for alpha, mse_val in zip(lasso.alphas, mse):
    print("Alpha: {:.2e} -- Mean squared error: {:.2f}".format(alpha, mse_val))

Alpha: 1.00e-10 -- Mean squared error: 0.01
Alpha: 1.00e-09 -- Mean squared error: 0.00
Alpha: 1.00e-08 -- Mean squared error: 0.00
Alpha: 1.00e-07 -- Mean squared error: 0.01
Alpha: 1.00e-06 -- Mean squared error: 0.00


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00010570123967501219, tolerance: 9.036588541666667e-05
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.838443311882203e-05, tolerance: 9.036588541666667e-05
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.735465930218318e-05, tolerance: 9.036588541666667e-05
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.p

In [18]:
#4.optimising kernel bandwidth using CV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
# Set up grid search to select bandwidth parameter
params = {'bandwidth': np.logspace(-1, 1, 20)}
grid = GridSearchCV(KernelDensity(), params, cv=5)
# Fit grid search on data and obtain optimal bandwidth
grid.fit(df1)
bw = grid.best_params_['bandwidth']
# Create KDE estimator with optimal bandwidth
kde = KernelDensity(bandwidth=bw)
# Fit KDE estimator on data
kde.fit(df1)

KernelDensity(bandwidth=0.12742749857031338)

In [19]:
#5.Use it to make predictions on new, unseen properties.
x1={'Size_(sq_ft)':2600,'Bedrooms':3,'Bathrooms':2,'Year_Built':2003,'Garage_Size':1,'Distance_to_School_(mi)':2.95}
df1 = df1.append(x1, ignore_index=True)

<ipython-input-19-5ede1f87af9a>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(x1, ignore_index=True)


In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Size_(sq_ft)             20 non-null     float64
 1   Bedrooms                 20 non-null     float64
 2   Bathrooms                20 non-null     float64
 3   Year_Built               20 non-null     float64
 4   Garage_Size              20 non-null     float64
 5   Distance_to_School_(mi)  20 non-null     float64
 6   Selling_Price_(k$)       19 non-null     float64
dtypes: float64(7)
memory usage: 1.2 KB


In [21]:
#5.Use it to make predictions on new, unseen properties.
model.predict([[2000,3,2,2003,2,1.5]])


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


array([0.42204914])